In [1]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, BertModel, BertForSequenceClassification, BertTokenizer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Concatenate, GlobalAveragePooling1D
import torch
from torchsummary import summary
import tensorflow as tf
import datasets
import numpy as np
import pandas as pd
from scipy.special import softmax
from sklearn import metrics
from sklearn.utils.class_weight import compute_class_weight
from time import time
import gc

## FRENK Dataset

In [2]:
def truncate(texts,max_length):
    for i in range(len(texts)):
      if len(texts[i]) > max_length:
        texts[i] = texts[i][:max_length]
    return texts

In [3]:
#FRENK dataset
ds = datasets.load_dataset("classla/FRENK-hate-en","binary")
frenk_train = ds["train"]["text"]
frenk_label_train = np.array(ds["train"]["label"])
frenk_test = ds["test"]["text"]
frenk_label_test = np.array(ds["test"]["label"])

Found cached dataset frenk-hate-en (C:/Users/Pedro/.cache/huggingface/datasets/classla___frenk-hate-en/binary/0.0.0/81373cea138e2fc4156010c45a19880095b891a74c39768895384d352bfe1ca0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
max_length = 150
frenk_train = truncate(frenk_train,max_length)
frenk_test = truncate(frenk_test,max_length)

In [5]:
print(np.array(frenk_test).shape)

(2301,)


In [6]:
frenk_cweights = compute_class_weight('balanced', classes=np.unique(frenk_label_train), y=frenk_label_train)
frenk_cweights = {0:frenk_cweights[0],1: frenk_cweights[1]}

## Gab Dataset

In [7]:
def get_texts(df):
    np_array = df["text"].to_numpy()
    texts = []
    for i in range(len(np_array)):
        texts.append(np_array[i])
    return texts

In [8]:
def get_hate_labels(labels):
    hate_labels = np.zeros(len(labels))
    for i in range(len(labels)):
        if labels[i].any():
            hate_labels[i] = 1
    return hate_labels

In [9]:
#Gab dataset
df_train = pd.read_csv('data/ghc_train.tsv',sep='\t')
ghc_train = get_texts(df_train)
labels = df_train[["hd","cv","vo"]].to_numpy()
ghc_label_train = get_hate_labels(labels)

df_test = pd.read_csv('data/ghc_test.tsv',sep='\t')
ghc_test = get_texts(df_test)
labels = df_test[["hd","cv","vo"]].to_numpy()
ghc_label_test = get_hate_labels(labels)

In [10]:
max_length = 150
ghc_train = truncate(ghc_train,max_length)
ghc_test = truncate(ghc_test,max_length)

In [11]:
ghc_cweights = compute_class_weight('balanced', classes=np.unique(ghc_label_train), y=ghc_label_train)
ghc_cweights = {0:ghc_cweights[0],1: ghc_cweights[1]}

## Twi-Red-You dataset

In [12]:
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   
df = dataset['train'].to_pandas()
df_hate = df[['text','hate_speech_score']]
df_hate['hate'] = (df_hate['hate_speech_score']>=0.5).astype(int)
df_hate = df_hate[['text','hate']]

df_hate_train = df_hate.iloc[0:8404]
df_hate_test = df_hate.iloc[8404:8404+2301]

df_hate_train.to_csv('data/try_train_8404.tsv', sep="\t", index=False)
df_hate_test.to_csv('data/try_test_2301.tsv', sep="\t", index=False)

Found cached dataset parquet (C:/Users/Pedro/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Pedro\AppData\Local\Temp\ipykernel_2040\1110734405.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hate['hate'] = (df_hate['hate_speech_score']>=0.5).astype(int)


In [13]:
#Twi-Red-You dataset
df_train = pd.read_csv('data/try_train_8404.tsv',sep='\t')
try_train = get_texts(df_train)
try_label_train = df_train['hate'].to_numpy()

df_test = pd.read_csv('data/try_test_2301.tsv',sep='\t')
try_test = get_texts(df_test)
try_label_test = df_test['hate'].to_numpy()

In [14]:
max_length = 150
try_train = truncate(try_train,max_length)
try_test = truncate(try_test,max_length)

In [15]:
try_cweights = compute_class_weight('balanced', classes=np.unique(try_label_train), y=try_label_train)
try_cweights = {0:try_cweights[0], 1:try_cweights[1]}

## Load models

In [16]:
#roBERTa models
sent_MOD = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
aggr_MOD = f"cardiffnlp/twitter-roberta-base-offensive"
gen_MOD = f"roberta-base"

#Load tokenizer and models
tokenizer = AutoTokenizer.from_pretrained(sent_MOD)
sent_mod = TFAutoModel.from_pretrained(sent_MOD)
aggr_mod = TFAutoModel.from_pretrained(aggr_MOD)
gen_mod = TFAutoModel.from_pretrained(gen_MOD)

batch_size = 32

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel

In [17]:
def Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out_1 = []
  batch_out_2 = []
  batch_out_3 = []
  for batch in batches:
      encoded_inputs = tokenizer(batch,padding=True,return_tensors='tf')
      # Compute the outputs of the modules as numpy arrays.
      emb_1 = sent_mod(encoded_inputs)[0]
      pool_1 = GlobalAveragePooling1D()(emb_1).numpy()
      batch_out_1.append(pool_1)

      emb_2 = aggr_mod(encoded_inputs)[0]
      pool_2 = GlobalAveragePooling1D()(emb_2).numpy()
      batch_out_2.append(pool_2)

      emb_3 = gen_mod(encoded_inputs)[0]
      pool_3 = GlobalAveragePooling1D()(emb_3).numpy()
      batch_out_3.append(pool_3)
    
      encoded_inputs = None
      emb_1 = None
      emb_2 = None
      emb_3 = None
      gc.collect() 
    
  outs_1 = np.concatenate(batch_out_1,axis=0)
  outs_2 = np.concatenate(batch_out_2,axis=0)
  outs_3 = np.concatenate(batch_out_3,axis=0)  

  return outs_1,outs_2,outs_3

## Compute outputs (If the outputs are already saved you can skip this step)

### FRENK dataset

In [15]:
#### Train set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,frenk_train,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/frenk/sent_train_outs.npy',outs_sent)
np.save('outs/frenk/aggr_train_outs.npy',outs_aggr)
np.save('outs/frenk/gen_train_outs.npy',outs_gen)

Elapsed time: 205.9961051940918


In [16]:
#### Test set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,frenk_test,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/frenk/sent_test_outs.npy',outs_sent)
np.save('outs/frenk/aggr_test_outs.npy',outs_aggr)
np.save('outs/frenk/gen_test_outs.npy',outs_gen)

Elapsed time: 71.29464769363403


### Gab dataset

In [17]:
#### Train set
batch_size = 32

t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,ghc_train,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/ghc/sent_train_outs.npy',outs_sent)
np.save('outs/ghc/aggr_train_outs.npy',outs_aggr)
np.save('outs/ghc/gen_train_outs.npy',outs_gen)

Elapsed time: 509.5066497325897


In [18]:
#### Test set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,ghc_test,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/ghc/sent_test_outs.npy',outs_sent)
np.save('outs/ghc/aggr_test_outs.npy',outs_aggr)
np.save('outs/ghc/gen_test_outs.npy',outs_gen)

Elapsed time: 137.6030786037445


### Twi-Red-You dataset

In [19]:
#### Train set
batch_size = 32

t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,try_train,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/try/sent_train_outs.npy',outs_sent)
np.save('outs/try/aggr_train_outs.npy',outs_aggr)
np.save('outs/try/gen_train_outs.npy',outs_gen)

Elapsed time: 167.61855840682983


In [20]:
#### Test set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,try_test,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/try/sent_test_outs.npy',outs_sent)
np.save('outs/try/aggr_test_outs.npy',outs_aggr)
np.save('outs/try/gen_test_outs.npy',outs_gen)

Elapsed time: 43.6887104511261


## Load outputs

In [18]:
frenk_sent_train = np.load('outs/frenk/sent_train_outs.npy')
frenk_aggr_train = np.load('outs/frenk/aggr_train_outs.npy')
frenk_gen_train = np.load('outs/frenk/gen_train_outs.npy')

frenk_sent_test = np.load('outs/frenk/sent_test_outs.npy')
frenk_aggr_test = np.load('outs/frenk/aggr_test_outs.npy')
frenk_gen_test = np.load('outs/frenk/gen_test_outs.npy')

In [19]:
ghc_sent_train = np.load('outs/ghc/sent_train_outs.npy')
ghc_aggr_train = np.load('outs/ghc/aggr_train_outs.npy')
ghc_gen_train = np.load('outs/ghc/gen_train_outs.npy')

ghc_sent_test = np.load('outs/ghc/sent_test_outs.npy')
ghc_aggr_test = np.load('outs/ghc/aggr_test_outs.npy')
ghc_gen_test = np.load('outs/ghc/gen_test_outs.npy')

In [20]:
try_sent_train = np.load('outs/try/sent_train_outs.npy')
try_aggr_train = np.load('outs/try/aggr_train_outs.npy')
try_gen_train = np.load('outs/try/gen_train_outs.npy')

try_sent_test = np.load('outs/try/sent_test_outs.npy')
try_aggr_test = np.load('outs/try/aggr_test_outs.npy')
try_gen_test = np.load('outs/try/gen_test_outs.npy')

In [21]:
print(try_sent_train.shape)

(8404, 768)


## Hate prediction with the 3 models

In [22]:
conc_frenk_train = np.concatenate([frenk_sent_train, frenk_aggr_train, frenk_gen_train], axis=1)
conc_frenk_test = np.concatenate([frenk_sent_test, frenk_aggr_test, frenk_gen_test], axis=1)

In [23]:
conc_ghc_train = np.concatenate([ghc_sent_train, ghc_aggr_train, ghc_gen_train], axis=1)
conc_ghc_test = np.concatenate([ghc_sent_test, ghc_aggr_test, ghc_gen_test], axis=1)

In [24]:
conc_ghc_train = conc_ghc_train[:8404]
conc_ghc_test = conc_ghc_test[:2301]

ghc_gen_train = ghc_gen_train[:8404]
ghc_gen_test = ghc_gen_test[:2301]

ghc_label_train = ghc_label_train[:8404]
ghc_label_test = ghc_label_test[:2301]

In [25]:
conc_try_train = np.concatenate([try_sent_train, try_aggr_train, try_gen_train], axis=1)
conc_try_test = np.concatenate([try_sent_test, try_aggr_test, try_gen_test], axis=1)

In [26]:
print(conc_try_train.shape)

(8404, 2304)


In [27]:
##Counterfactual evaluation
def evaluate_text_PEACE(text,model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size):
    out_sent,out_aggr,out_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,text,batch_size)
    conc = np.concatenate([out_sent,out_aggr,out_gen],axis=1)
    prob = model.predict(conc)
    print("Hate prob.:",prob)

In [28]:
n_avg = 10

### Training with FRENK

In [29]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_ghc = []
f1_test_ghc = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_frenk_train, frenk_label_train, class_weight=frenk_cweights, epochs=5, batch_size=32)
    #class_weight=frenk_cweights

    probs = hate_model.predict(conc_frenk_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,frenk_label_train))
    f1_train.append(metrics.f1_score(preds,frenk_label_train,zero_division=1))

    probs = hate_model.predict(conc_frenk_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_ghc_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_try_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.5351 - accuracy: 0.7249
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4943 - accuracy: 0.7458
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4767 - accuracy: 0.7550
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4615 - accuracy: 0.7629
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4514 - accuracy: 0.7724
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5272 - accuracy: 0.7283
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4952 - accuracy: 0.7473
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4785 - accuracy: 0.7570
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4635 - accuracy: 0.7644
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4461 - accuracy: 0.7689
Epoch 1/5


In [30]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.7766420752022847
Avg. f1 score: 0.7292583514331002


In [31]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

FRENK Test set:
Avg. accuracy: 0.7407648848326814
Avg. f1 score: 0.6956710804190428


In [32]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.6405041286397218
Avg. f1 score: 0.3574528174062537


In [33]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.5627987831377662
Avg. f1 score: 0.5434445857352348


In [34]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_PEACE(text,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_PEACE(text_aggr,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.43836534]]
Hate prob.: [[0.7437929]]


### Training with Gab

In [35]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_ghc_train, ghc_label_train, class_weight=ghc_cweights, epochs=5, batch_size=32)

    probs = hate_model.predict(conc_ghc_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,ghc_label_train))
    f1_train.append(metrics.f1_score(preds,ghc_label_train,zero_division=1))

    probs = hate_model.predict(conc_ghc_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_frenk_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_try_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4825 - accuracy: 0.7764
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4302 - accuracy: 0.7976
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4051 - accuracy: 0.8024
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.3983 - accuracy: 0.8053
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3766 - accuracy: 0.8119
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4733 - accuracy: 0.7790
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4226 - accuracy: 0.7989
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4123 - accuracy: 0.7999
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3925 - accuracy: 0.8051
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3828 - accuracy: 0.8109
Epoch 1/5


In [36]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8179914326511184
Avg. f1 score: 0.5473030254917081


In [37]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Gab Test set:
Avg. accuracy: 0.7820947414167754
Avg. f1 score: 0.47888882278551365


In [38]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.7211647109952195
Avg. f1 score: 0.6325809920064056


In [39]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.5084745762711863
Avg. f1 score: 0.5165592099162692


In [40]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_PEACE(text,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_PEACE(text_aggr,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.1208026]]
Hate prob.: [[0.2779474]]


### Training with Twi-Red-You

In [41]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_ghc = []
f1_test_ghc = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_try_train, try_label_train, class_weight=try_cweights, epochs=5, batch_size=32)

    probs = hate_model.predict(conc_try_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,try_label_train))
    f1_train.append(metrics.f1_score(preds,try_label_train,zero_division=1))
    
    probs = hate_model.predict(conc_try_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,try_label_test))
    f1_test.append(metrics.f1_score(preds,try_label_test,zero_division=1))

    probs = hate_model.predict(conc_frenk_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_ghc_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.4914 - accuracy: 0.7450
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4585 - accuracy: 0.7650
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4399 - accuracy: 0.7783
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4344 - accuracy: 0.7774
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4218 - accuracy: 0.7877
Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.4924 - accuracy: 0.7473
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4566 - accuracy: 0.7629
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4447 - accuracy: 0.7701
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4307 - accuracy: 0.7843
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4213 - accuracy: 0.7838
Epoch 1/5


In [42]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.7949904807234649
Avg. f1 score: 0.6834308282605249


In [43]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Twi-Red-You Test set:
Avg. accuracy: 0.7715775749674053
Avg. f1 score: 0.6579756774653944


In [44]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6830508474576271
Avg. f1 score: 0.36303836264372125


In [45]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.8308561495002174
Avg. f1 score: 0.34196266018199906


In [46]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_PEACE(text,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_PEACE(text_aggr,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.15771471]]
Hate prob.: [[0.2806684]]


## Hate prediction with the general model

In [47]:
##Counterfactual evaluation
def evaluate_text_gen(text,model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size):
    _,_,out_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,text,batch_size)
    prob = model.predict(out_gen)
    print("Hate prob.:",prob)

### Training with FRENK

In [48]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_ghc = []
f1_test_ghc = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model_gen = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model_gen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model_gen.fit(frenk_gen_train, frenk_label_train, class_weight=frenk_cweights, epochs=5, batch_size=32)

    probs = hate_model_gen.predict(frenk_gen_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,frenk_label_train))
    f1_train.append(metrics.f1_score(preds,frenk_label_train,zero_division=1))

    probs = hate_model_gen.predict(frenk_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(ghc_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(try_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5869 - accuracy: 0.6663
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5318 - accuracy: 0.7178
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5135 - accuracy: 0.7293
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4940 - accuracy: 0.7421
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4781 - accuracy: 0.7523
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5958 - accuracy: 0.6631
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5321 - accuracy: 0.7223
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5118 - accuracy: 0.7356
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4909 - accuracy: 0.7464
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4729 - accuracy: 0.7594
Epoch 1/5


In [49]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.7823417420276059
Avg. f1 score: 0.715327504980199


In [50]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

FRENK Test set:
Avg. accuracy: 0.7416775315080399
Avg. f1 score: 0.6767199980036148


In [51]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.6256410256410255
Avg. f1 score: 0.3383597182992352


In [52]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.5848326814428508
Avg. f1 score: 0.5418748871750005


In [53]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_gen(text,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_gen(text_aggr,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.30210936]]
Hate prob.: [[0.5940976]]


### Training with Gab

In [54]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model_gen = Sequential([
          Flatten(),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model_gen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model_gen.fit(ghc_gen_train, ghc_label_train, class_weight=ghc_cweights, epochs=5, batch_size=32)

    probs = hate_model_gen.predict(ghc_gen_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,ghc_label_train))
    f1_train.append(metrics.f1_score(preds,ghc_label_train,zero_division=1))

    probs = hate_model_gen.predict(ghc_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(frenk_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(try_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5682 - accuracy: 0.6944
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4809 - accuracy: 0.7678
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4589 - accuracy: 0.7784
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4415 - accuracy: 0.7864
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4310 - accuracy: 0.7872
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5591 - accuracy: 0.7113
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4806 - accuracy: 0.7653
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4568 - accuracy: 0.7836
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4426 - accuracy: 0.7884
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4315 - accuracy: 0.7943
Epoch 1/5


In [55]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8051404093288911
Avg. f1 score: 0.5126210489476654


In [56]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Gab Test set:
Avg. accuracy: 0.7764450239026509
Avg. f1 score: 0.46506978348370065


In [57]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.7115167318557148
Avg. f1 score: 0.6230684694241557


In [58]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.5088657105606258
Avg. f1 score: 0.5083455918478467


In [59]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_gen(text,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_gen(text_aggr,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.08308121]]
Hate prob.: [[0.7219449]]


### Training with Twi-Red-You

In [60]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_ghc = []
f1_test_ghc = []
for i in range(n_avg):
    hate_model_gen = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model_gen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model_gen.fit(try_gen_train, try_label_train, class_weight=try_cweights, epochs=5, batch_size=32)

    probs = hate_model_gen.predict(try_gen_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,try_label_train))
    f1_train.append(metrics.f1_score(preds,try_label_train,zero_division=1))
    
    probs = hate_model_gen.predict(try_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,try_label_test))
    f1_test.append(metrics.f1_score(preds,try_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(frenk_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))

    probs = hate_model_gen.predict(ghc_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.5355 - accuracy: 0.7147
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4803 - accuracy: 0.7427
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4666 - accuracy: 0.7525
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4517 - accuracy: 0.7580
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4363 - accuracy: 0.7706
Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.5324 - accuracy: 0.7186
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4833 - accuracy: 0.7399
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4658 - accuracy: 0.7532
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4525 - accuracy: 0.7575
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4424 - accuracy: 0.7639
Epoch 1/5


In [61]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.7621846739647786
Avg. f1 score: 0.6656105256370761


In [62]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Twi-Red-You Test set:
Avg. accuracy: 0.7315080399826162
Avg. f1 score: 0.628906796155898


In [63]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6758365927857453
Avg. f1 score: 0.46327030801835967


In [64]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.7678835289004781
Avg. f1 score: 0.34375756808165686


In [65]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_gen(text,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_gen(text_aggr,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.14943363]]
Hate prob.: [[0.73269844]]


## HateBERT

In [66]:
model_dir = "HateBERT_hateval"
HateBERT = BertModel.from_pretrained(
    model_dir # Use pre-trained model from its directory, change this to use a pre-trained model from bert
)
HateBERT.eval()
HateBERT_tok = BertTokenizer.from_pretrained(model_dir)

Some weights of the model checkpoint at HateBERT_hateval were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [67]:
def Comp_Embed_HateBERT(tokenizer,mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out = []

  with torch.no_grad():
      for batch in batches:
          encoded_inputs = tokenizer(batch,padding=True,return_tensors='pt')
          # Compute the output of the module as a numpy array.

          emb = mod(**encoded_inputs)[0]
          tf_emb = tf.convert_to_tensor(emb.numpy())  
          pool = GlobalAveragePooling1D()(tf_emb).numpy()
          batch_out.append(pool)

          encoded_inputs = None
          tf_emb = None
          gc.collect() 
    
  outs = np.concatenate(batch_out,axis=0)
  return outs

## Compute HateBERT embeddings (If these outputs are already saved you can skip this step)

### FRENK dataset

In [57]:
###Train set
batch_size = 32

t = time()
outs_train = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,frenk_train,batch_size)
print("Elapsed time:",time()-t)
print(outs_train.shape)

np.save('outs/frenk/HateBERT_train_emb.npy',outs_train)

Elapsed time: 482.1618766784668
(8404, 768)


In [58]:
###Test set
t = time()
outs_test = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,frenk_test,batch_size)
print("Elapsed time:",time()-t)
print(outs_test.shape)

np.save('outs/frenk/HateBERT_test_emb.npy',outs_test)

Elapsed time: 129.07557344436646
(2301, 768)


### Gab dataset

In [59]:
###Train set
batch_size = 32

t = time()
outs_train = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,ghc_train,batch_size)
print("Elapsed time:",time()-t)
print(outs_train.shape)

np.save('outs/ghc/HateBERT_train_emb.npy',outs_train)

Elapsed time: 1384.8847632408142
(22036, 768)


In [60]:
###Test set
t = time()
outs_test = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,ghc_test,batch_size)
print("Elapsed time:",time()-t)
print(outs_test.shape)

np.save('outs/ghc/HateBERT_test_emb.npy',outs_test)

Elapsed time: 339.3550543785095
(5510, 768)


### Twi-Red-You dataset

In [61]:
###Train set
batch_size = 32

t = time()
outs_train = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,try_train,batch_size)
print("Elapsed time:",time()-t)
print(outs_train.shape)

np.save('outs/try/HateBERT_train_emb.npy',outs_train)

Elapsed time: 464.42991185188293
(8404, 768)


In [62]:
###Test set
t = time()
outs_test = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,try_test,batch_size)
print("Elapsed time:",time()-t)
print(outs_test.shape)

np.save('outs/try/HateBERT_test_emb.npy',outs_test)

Elapsed time: 125.01945090293884
(2301, 768)


## Hate prediction with HateBERT

In [68]:
frenk_train_emb = np.load('outs/frenk/HateBERT_train_emb.npy')
frenk_test_emb = np.load('outs/frenk/HateBERT_test_emb.npy')

In [69]:
ghc_train_emb = np.load('outs/ghc/HateBERT_train_emb.npy')
ghc_test_emb = np.load('outs/ghc/HateBERT_test_emb.npy')

In [70]:
ghc_train_emb = ghc_train_emb[:8404]
ghc_test_emb = ghc_test_emb[:2301]

In [71]:
try_train_emb = np.load('outs/try/HateBERT_train_emb.npy')
try_test_emb = np.load('outs/try/HateBERT_test_emb.npy')

In [72]:
##Counterfactual evaluation
def evaluate_text_HateBERT(text,model,tokenizer,HateBERT,batch_size):
    out = Comp_Embed_HateBERT(tokenizer,HateBERT,text,batch_size)
    prob = model.predict(out)
    print("Hate prob.:",prob)

### Training with FRENK

In [73]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_ghc = []
f1_test_ghc = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(frenk_train_emb, frenk_label_train, class_weight=frenk_cweights, epochs=5, batch_size=32)

    probs = classifier_HateBERT.predict(frenk_train_emb)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,frenk_label_train))
    f1_train.append(metrics.f1_score(preds,frenk_label_train,zero_division=1))

    probs = classifier_HateBERT.predict(frenk_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(ghc_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(try_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.5596 - accuracy: 0.7004
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4986 - accuracy: 0.7400
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4554 - accuracy: 0.7750
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4108 - accuracy: 0.8043
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.3521 - accuracy: 0.8344
Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.5578 - accuracy: 0.6981
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4964 - accuracy: 0.7471
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4504 - accuracy: 0.7774
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4049 - accuracy: 0.8068
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.3615 - accuracy: 0.8302
Epoch 1/5


In [74]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.870168967158496
Avg. f1 score: 0.8332237738193307


In [75]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

FRENK Test set:
Avg. accuracy: 0.7372012168622339
Avg. f1 score: 0.6701161671842291


In [76]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.686484137331595
Avg. f1 score: 0.358560002722806


In [77]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.5626684050412863
Avg. f1 score: 0.5265644938584841


In [78]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_HateBERT(text,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)
evaluate_text_HateBERT(text_aggr,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)

Hate prob.: [[0.49056503]]
Hate prob.: [[0.81344473]]


### Training with Gab

In [79]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(ghc_train_emb, ghc_label_train, class_weight=ghc_cweights, epochs=5, batch_size=32)

    probs = classifier_HateBERT.predict(ghc_train_emb)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,ghc_label_train))
    f1_train.append(metrics.f1_score(preds,ghc_label_train,zero_division=1))

    probs = classifier_HateBERT.predict(ghc_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(frenk_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(try_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5307 - accuracy: 0.7537
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4729 - accuracy: 0.7840
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4439 - accuracy: 0.7878
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4191 - accuracy: 0.8007
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3981 - accuracy: 0.8126
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5272 - accuracy: 0.7454
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4662 - accuracy: 0.7842
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4435 - accuracy: 0.7880
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4136 - accuracy: 0.8063
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3849 - accuracy: 0.8219
Epoch 1/5


In [80]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.78501903855307
Avg. f1 score: 0.5259652440104791


In [81]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Gab Test set:
Avg. accuracy: 0.7402433724467622
Avg. f1 score: 0.44014339573324285


In [82]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.684832681442851
Avg. f1 score: 0.6422986514749863


In [83]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.46484137331594966
Avg. f1 score: 0.4982154721255137


In [84]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_HateBERT(text,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)
evaluate_text_HateBERT(text_aggr,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)

Hate prob.: [[0.0921497]]
Hate prob.: [[0.8318603]]


### Training with Twi-Red-You 

In [85]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_ghc = []
f1_test_ghc = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(try_train_emb, try_label_train, class_weight=try_cweights, epochs=5, batch_size=32)

    probs = classifier_HateBERT.predict(try_train_emb)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,try_label_train))
    f1_train.append(metrics.f1_score(preds,try_label_train,zero_division=1))
    
    probs = classifier_HateBERT.predict(try_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,try_label_test))
    f1_test.append(metrics.f1_score(preds,try_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(frenk_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))

    probs = classifier_HateBERT.predict(ghc_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.5024 - accuracy: 0.7367
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4484 - accuracy: 0.7732
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4127 - accuracy: 0.7870
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.3817 - accuracy: 0.8068
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.3447 - accuracy: 0.8278
Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.5022 - accuracy: 0.7352
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4544 - accuracy: 0.7614
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4253 - accuracy: 0.7861
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.3909 - accuracy: 0.8094
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.3545 - accuracy: 0.8277
Epoch 1/5


In [86]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8453474535935269
Avg. f1 score: 0.7645656065362127


In [87]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Twi-Red-You Test set:
Avg. accuracy: 0.760799652325076
Avg. f1 score: 0.6451853010772671


In [88]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6785745328118209
Avg. f1 score: 0.3796600664283095


In [89]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.82290308561495
Avg. f1 score: 0.34940647569454697


In [90]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_HateBERT(text,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)
evaluate_text_HateBERT(text_aggr,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)

Hate prob.: [[0.00160116]]
Hate prob.: [[0.73066837]]
